# 🧠 The Illusion of Memory in Chat-Based LLMs

When interacting with ChatGPT or similar systems, it feels like the model remembers what we said earlier.

We say our name once, ask a follow-up question later, and the model answers correctly. Creating so a strong illusion of memory. In reality, Large Language Models have no memory at all.

This notebook demonstrates why.

---

## 🔍 A key idea to keep in mind
> Every API call to an LLM is completely stateless.

The model: does not remember previous calls, does not store user information, does not know what happened “before” unless we explicitly tell it.
Each request is handled independently.

---

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


## 🧪 First experiment: a single-turn interaction
Let’s start with a simple message exchange where the model responds politely and may even greet us by name.

In [2]:
from openai import OpenAI

openai = OpenAI()
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hello There! I'm Stella"}
    ]
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'Hello Stella! How can I assist you today?'

## ❓ Second experiment: a follow-up question
Now we ask a second question in a new API call, but we do not include the previous message.
Here the models replies with something like:
```text
I’m sorry, but I don’t know your name.
```

In [3]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What's my name?"}
    ]

response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'I’m sorry, but I don’t know your name. Could you please tell me?'

## 🤯 What just happened?

Nothing went wrong. Indeed, this behavior is **correct**.
> The model did not forget your name — it never knew it in the first place.

Why? Because:
- the second API call did not include the earlier message
- the model only sees the input it receives **right now**.

---

## 🧩 How chat systems create the illusion of memory
Chat-based applications (like ChatGPT) use a simple but powerful trick:
> They resend the entire conversation history at every turn.
>
Instead of sending only the latest user message, they send: the system prompt, all previous user messages, all previous assistant responses, and the new user message.
All of this becomes one long input sequence.

---

## 🔁 Recreating “memory” manually
Let’s try again, this time including the full conversation.

In [4]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Stella!"},
    {"role": "assistant", "content": "Hi Stella! How can I assist you today?"},
    {"role": "user", "content": "What's my name?"}
    ]
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'Your name is Stella! How can I help you today?'

## 💸 Why context affects cost
Because the entire conversation is resent every time:
- longer conversations → more input tokens
- more input tokens → more computation
- more computation → higher cost
-
This is not a bug — it’s the price of maintaining context.
You are literally asking the model to reprocess the entire past conversation at every turn.

---

## 🔗 Connection with tokenization
This notebook connects directly to what we learned earlier:
- conversation history = more tokens
- more tokens = higher cost
t- oken limits define how much “memory” you can simulate.

In practice:
> LLM memory is just tokenized text that you keep sending back.